In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pickle
from keras.models import load_model
import os 


with open('../saved_model/ens_model.pkl', 'rb') as file:
    ens_model = pickle.load(file)
with open('../saved_model/knn_model.pkl', 'rb') as file:
    knn_model = pickle.load(file)
with open('../saved_model/svm_model.pkl', 'rb') as file:
    svm_model = pickle.load(file)
nn_model = load_model('../saved_model/nn_model.keras')  # Assuming load_model loads the neural network


2024-05-20 00:03:21.209304: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-20 00:03:21.244613: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 00:03:21.933578: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('/home/khangpt/MUSIC-GEN-PROJ/GTZAN/Data/features_30_sec.csv')
def preprocess_(df):
    df = df.iloc[0:,1:]
    X = df.drop(['length','label'], axis = 1)
    y = df['label']

    df.label = pd.Categorical(df.label)
    y = np.array(df.label.cat.codes)
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    return X, y
X, y = preprocess_(df)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y, random_state=42)

In [3]:
y_pred =  nn_model.predict(X_test).argmax(axis=-1)
print(accuracy_score(y_test,y_pred))

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
0.76


In [4]:
y_pred =  svm_model.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.735


In [5]:
y_pred =  knn_model.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.745


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Input
from scikeras.wrappers import KerasClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

# Define the KNN classifier
KNN = KNeighborsClassifier(n_neighbors=6, weights='distance')

# Define the SVM classifier
SVM = SVC(kernel='rbf', gamma='scale', C=11, probability=True)

# Define the neural network model
def create_nn():
    model = Sequential([
        Input(shape=(57,)), 
        Dense(300, input_dim=57,activation='relu'),                                                                      
        Dense(300, activation='relu'),
        Dense(300, activation='relu'),
        Dense(10, activation='softmax')
    ])
    adam = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=adam,
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

NN = KerasClassifier(build_fn=create_nn, epochs=150, batch_size=32, verbose=0)

voting = VotingClassifier(estimators=[('knn', KNN), ('svm', SVM), ('nn', NN)],
                          voting='soft')


In [7]:
KNN.fit(X_train, y_train)
y_pred_KNN = KNN.predict(X_test)
print(accuracy_score(y_pred_KNN,y_test))

0.745


In [8]:
SVM.fit(X_train, y_train)
y_pred_SVM = SVM.predict(X_test)
print(accuracy_score(y_pred_SVM,y_test))

0.77


In [9]:
NN.fit(X_train, y_train)
y_pred_NN = NN.predict(X_test)
print(accuracy_score(y_pred_NN,y_test))

/home/khangpt/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/khangpt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


0.73


In [10]:
voting.fit(X_train, y_train)
y_pred_vote = voting.predict(X_test)
print(accuracy_score(y_pred_vote,y_test))

/home/khangpt/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/khangpt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


0.765


In [11]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_KNN))

              precision    recall  f1-score   support

           0       0.83      0.75      0.79        20
           1       0.87      1.00      0.93        20
           2       0.75      0.90      0.82        20
           3       0.56      0.50      0.53        20
           4       0.75      0.75      0.75        20
           5       0.88      0.75      0.81        20
           6       0.88      0.75      0.81        20
           7       0.73      0.80      0.76        20
           8       0.67      0.60      0.63        20
           9       0.57      0.65      0.60        20

    accuracy                           0.74       200
   macro avg       0.75      0.74      0.74       200
weighted avg       0.75      0.74      0.74       200



In [12]:
print(classification_report(y_test, y_pred_SVM))

              precision    recall  f1-score   support

           0       0.78      0.90      0.84        20
           1       0.95      0.95      0.95        20
           2       0.80      0.80      0.80        20
           3       0.65      0.55      0.59        20
           4       0.72      0.90      0.80        20
           5       0.74      0.85      0.79        20
           6       1.00      0.75      0.86        20
           7       0.82      0.70      0.76        20
           8       0.65      0.65      0.65        20
           9       0.65      0.65      0.65        20

    accuracy                           0.77       200
   macro avg       0.78      0.77      0.77       200
weighted avg       0.78      0.77      0.77       200



In [13]:
print(classification_report(y_test, y_pred_NN))

              precision    recall  f1-score   support

           0       0.75      0.60      0.67        20
           1       0.95      0.90      0.92        20
           2       0.68      0.75      0.71        20
           3       0.55      0.55      0.55        20
           4       0.64      0.70      0.67        20
           5       0.72      0.90      0.80        20
           6       0.94      0.85      0.89        20
           7       0.85      0.85      0.85        20
           8       0.67      0.70      0.68        20
           9       0.59      0.50      0.54        20

    accuracy                           0.73       200
   macro avg       0.73      0.73      0.73       200
weighted avg       0.73      0.73      0.73       200



In [14]:
print(classification_report(y_test, y_pred_vote))

              precision    recall  f1-score   support

           0       0.76      0.65      0.70        20
           1       0.91      1.00      0.95        20
           2       0.81      0.85      0.83        20
           3       0.83      0.50      0.62        20
           4       0.71      0.85      0.77        20
           5       0.85      0.85      0.85        20
           6       1.00      0.80      0.89        20
           7       0.76      0.80      0.78        20
           8       0.60      0.75      0.67        20
           9       0.55      0.60      0.57        20

    accuracy                           0.77       200
   macro avg       0.78      0.76      0.76       200
weighted avg       0.78      0.77      0.76       200



In [15]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
stacking = StackingClassifier(estimators=[('knn', KNN), ('svm', SVM), ('nn', NN)],
                              final_estimator=LogisticRegression())

In [16]:
stacking.fit(X_train, y_train)
y_pred_stack = stacking.predict(X_test)
print(accuracy_score(y_test, y_pred_stack))

/home/khangpt/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/khangpt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/khangpt/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/khangpt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to 

/home/khangpt/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/khangpt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/khangpt/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/khangpt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to 

0.775
